# Tutorial n°3 - SDK & Pollen-Vision : Reachy the greengrocer

In this tutorial, we will learn how to do tasks with Reachy using :
- the SDK client to make it move,
- the module pollen-vision to use object detection in its environment. 

Here, we are going to ask Reachy to sort fruits on a table and to drop them in user-defined places, according to which fruit it is. 

What you will learn : 
- How to do object detection
- How to switch from the image frame to the robot frame
- How to make Reachy move according to what it sees

## 1. Prerequisites

To use the SDK client and pollen-vision, you first need to install them. If it is not the case, checkout the section below 👇
<details>

<summary>Install the Python library reachy2-sdk</summary>

In general, you'd better **work in a virtual environment**. You have 2 different ways to install the sdk : 
- by running the following command:

<code>
pip install reachy2-sdk -e .
</code>

- from source by following the instructions on the [GitHub repository](https://github.com/pollen-robotics/reachy2-sdk)

</details>

<details>

<summary>Install the Python library pollen-vision</summary>

In general, you'd better **work in a virtual environment**. You have 2 different ways to install the sdk : 
- by running the following command:

<code>
pip install pollen-vision
</code>

- from source by following the instructions on the [GitHub repository](https://github.com/pollen-robotics/pollen-vision)

</details>

If you have never used the **SDK client** before, don't forget to do the [Getting Started notebooks](https://github.com/pollen-robotics/reachy2-sdk/tree/develop/src/examples), that will help you understand all the basics you need to know about it ! And if you want to go further, you can also check the [SDK documentation](https://pollen-robotics.github.io/reachy2-sdk/reachy2_sdk.html). 

You can also familiarize yourself with **pollen-vision** by making the [Vision Models notebooks](https://github.com/pollen-robotics/pollen-vision/tree/develop/examples/vision_models_examples). 


## 2. Set up

### 2.1. Material

For this tutorial, you have to set up your environment. 
So you need :
- Reachy (obviously)
- a table, no higher than the robot's elbows
- a white plate
- a white bowl
- apples
- oranges


### 2.2. Scene

> Your Reachy must be at a sufficient **height** so that its outstretched arms do not touch the mobile base. 

> As usual, Reachy needs to be in a safe environment with enough place to move around, no one in reachable space and no obstacles. And always keep the emergency stop button nearby!

To set up your scene : 

1.    Switch Reachy on and place it facing the table but 20 cm back (*it will need to bent its arms without obstacle, then it will be moved forward to the edge of the table*). 
2.    Make sure the torso camera is connected to the bedrock, as that's the one we'll be using here (*it's the one pluged on the outermost USB port*).
3.    On the table, place the white plate on the left, then the oranges, the apples and the white bowl, such as the image below. Don't put them too far, Reachy doesn't have expandable arms.
4.    You will need good lighting so that objects can be detected.

<p align="center"> 
    <img src="images/set_up_tuto_profil.jpg" alt="Image 1" style="width: 22.5%; margin-right: 10px;"/>
    <img src="images/set_up_tuto_face.jpg" alt="Image 2" style="width: 40%;"/>
</p>



When you're all set up, you can move on to the next step.

## 3. Preview

In this tutorial, we'll gradually build the program that will enable Reachy to sort fruit using the SDK Client and Pollen-Vision. This is what it your Reachy will be able to do at the end ! 

<p align = "center"> 
    <img src="images/gif_oranges.gif" alt="Gif 1" style="width: 27.5%; margin-right: 10px;"/>
    <img src="images/gif_apple_human.gif" alt="Gif 2" style="width: 40%; "/>
</p>

Exciting, isn't it ? So let's get to work !

## 4. Let's build it ! 

### 4.1. Instanciation of the SDK

First, we will connect to the robot. To do so, we need to import the SDK client package and to instanciate a ReachySDK. That module is the one allowing us to control Reachy.

Two requirements :
- Your computer needs to be on the same network as the robot.
- You need to know your Reachy's IP : to do so, you have two options : 
    - you can check it on the dashboard (*.local_IP_address_of_your_robot:8000* in a browser), section Network. 
    - you can have a look at the small screen on Reachy's back, that will show you one at a time its Ethernet and its Wifi IP. 

Now, let's connect to it. 

In [ ]:
#import the package
from reachy2_sdk import ReachySDK 

#connect to the robot
reachy = ReachySDK('localhost') # replace 'localhost' with the actual IP address of your Reachy
print("Reachy is connected :", reachy.is_connected())

If you are getting the message "Could not connect to Reachy", make sure that :
-  Reachy is turned on
- the reachy2_core.service is running 
- the IP address is correct. 

*More info on the debug section of the [sdk documentation](https://pollen-robotics.github.io/reachy2-docs/help/help/recovering/).<br>*

### 4.2. Instanciation of Pollen-Vision 

Then, we will use Pollen-Vision repository to be able to detect objects from the camera view thanks to AI and extract informations about them, for Reachy to adapt its actions to its environment.

More specifically, we'll be using the *Perception* module, which combines object detection using a [YOLO model](https://docs.ultralytics.com/models/yolo-world/), image segmentation and conversion of the extracted informations from pixels into real-world coordinates. 


#### 4.2.1. How does it work

We set the name of the objects we're interested in, and the model detects those objects with a prediction confidence score (from 0, the lowest, to 1, the highest). And from those detections, we get different informations about the objects such as the coordinates of the bounding box, the centroid, and the depth. As this is done on Reachy's camera view, we can track the objects dynamically and minimize the impact of artifacts by retaining only the objects that are always detected from one frame to the next. 

Then, we can switch from pixels frame to Reachy's frame and get the informations of the objects in the real world frame, using parameters such as the camera intrinsic matrix (that allows to transform pixel coordinates into 3D coordinates from the camera frame) and the transform of the camera in Reachy'sframe (*T_reachy_cam*, which is known and allows to switch from the camera to Reachy's frame of reference). 


#### 4.2.2. How to instanciate it

To define it, we need to get all the informations we talked about before : 

- the intrinsic matrix of the camera, that can be found thanks to the PollenSDKCameraWrapper, a class of pollen-vision that allows to get informations about Reachy's cameras. 
- the T_reachy_cam
- the prediction score threshold for the objects to be considered
- the frequency at which we want to perform object detection on the camera frames

##### **PollenSDKCameraWrapper**

We are going to use this module to get informations about Reachy's camera, such as : 
- the images captured by the camera, with *get_data()* 
- parameters like the camera intrinsic matrix (named K by convention) with *get_K()* for the RGB camera, and *get_depth_K()* for the depth camera. 

We need to instanciate it, with the SDK client object we instanciated earlier. Then, we are going to print the image captured by the torso camera when you execute the cell. 

In [ ]:
from pollen_vision.camera_wrappers.pollen_sdk_camera.pollen_sdk_camera_wrapper import PollenSDKCameraWrapper

# instanciation of the camera wrapper
r_cam = PollenSDKCameraWrapper(reachy)

# get the image as a np.array, and the timestamp of the image
data, _, timestamp = r_cam.get_data()

# displays the image from the RGB camera (depth camera is available by changing 'left' to 'depth')
from PIL import Image
img = data['left']
Image.fromarray(img) 

##### **T_reachy_cam**

Now, we need to define the T_reachy_cam. The camera frame is oriented differently from the robot frame, with the x axis to the left, the y axis down and the z axis back (as a reminder, the robot frame of reference has the x axis forward, the y axis to the left and the z axis up). Moreover, the torso camera is tilted downwards. 

In [ ]:
from reachy2_sdk.utils.utils import invert_affine_transformation_matrix

T_cam_reachy = reachy.cameras.depth.get_extrinsics()
T_reachy_cam = invert_affine_transformation_matrix(T_cam_reachy)

##### **Labels** 

To know what to use as a frequency and prediction score threshold, you first need to set the labels you're gonna use to say what you want to detect. Indeed, to use IA model for object detection, it's important to find the most relevant label for the model to find the objects, and it's not always the most intuitive words. 

To test those, you can use directly two submodules used in Perception to see if your objects are correctly detected on your image :
-  the YOLO model (*the one that detects objects*) 
-  the Annotator (*the one that allows the visualization*).

Be creative, try even non-instinctive words, with more or less precise adjectives, and test everything you can ! 

In this tutorial, we decide to define two objects in which to place the fruit: a right-hand side and a left-hand side. And we'll define the type of fruit to be placed on the right and the one on the left. Here, we have a plate on the left and a bowl on the right, and the fruits to be placed are oranges and apples.

We are going to try classic labels for the detection, in the candidate_labels, but you'll see that we need to adjust those because the detection won't be good enough. 

*Don't worry if the instanciation of the YoloWorldWrapper is taking a while, it can take about 1-1.5min to create it the first time. And you can have a warning message, with no impact on the rest of the tutorial*.

In [ ]:
from pollen_vision.vision_models.object_detection import YoloWorldWrapper
from pollen_vision.utils import get_bboxes
from pollen_vision.utils import Annotator

#those first labels won't be good enough
labels = ["orange", "apple", "plate", "bowl"]

yolo = YoloWorldWrapper() #allows to use the YOLO model for object detection
annotator = Annotator() #allows to get annotated image with the detected objects
yolo_predictions = yolo.infer(im=img, candidate_labels=labels, detection_threshold=0.15) 
bboxes = get_bboxes(yolo_predictions) #returns the bounding boxes of the detected objects
img_annotated = annotator.annotate(im=img, detection_predictions=yolo_predictions) 
Image.fromarray(img_annotated)


Here, you could see that the oranges aren't detected as "orange" and may be detected as apples. We are going to try new labels to be more effective. Also, you can add adjectives to be sure not to have any false positive. 

In [ ]:
labels = ["orange ball", "apple", "white plate", "white bowl"]
yolo_predictions = yolo.infer(im=img, candidate_labels=labels, detection_threshold=0.15)
bboxes = get_bboxes(yolo_predictions)
img_annotated = annotator.annotate(im=img, detection_predictions=yolo_predictions)
Image.fromarray(img_annotated)

Now, we have good labels to detect our objects. 


##### **Detection threshold & Frequency**

Now that we have our labels, we can see that we also have the prediction scores on the image : that will help you set the detection threshold when you instanciate your Perception object. Here, we decide to set it to 0.2, because the oranges are detected with a low score. 

The frequency depends on how well your objects are detected (if they're not well detected, a higher frequency will increase the number of frames over which the model attempts detection). Here, we are going to set it to 40 because the oranges aren't detected often. 

#### 4.2.3. Instanciation

Now we have all the elements to instanciate the Perception object : the CameraWrapper, the T_reachy_cam, the frequency of detection and the detection threshold.

In [ ]:
from pollen_vision.perception import Perception

perception = Perception(r_cam, T_reachy_cam, freq=40, yolo_thres=0.2)

print("The Perception object is created.")

And now, we can start the objects detection and the tracking: we start by setting the tracked objects, then we launch it. 

You can display the image from the camera continuously, together with the objects detected, their bounding boxes and their masks, the prediction scores and their centroids, thanks to the "visualize=True".

In [ ]:
left_obj_target, right_obj_target = "white plate", "white bowl"
obj_to_left, obj_to_right = "orange ball", "apple"

perception.set_tracked_objects([obj_to_left, obj_to_right, left_obj_target, right_obj_target])
perception.start(visualize=True)

You have now a window with the camera view and the detected objects. The Perception part is now on.

> *If you didn't get the new window, the notebook crashed unfortunately. Please restart your kernel and execute again the cells above.*

### 4.3. Set Reachy ready

We want Reachy to be in front of the table where there are the fruits, in a position that allows it to access the table without obstructing its camera's view. 

Reachy has its motors off by default. So first, we need to turn them on. 

> Put Reachy in a safe environment, with no one in reachable space and no obstacles, and with someone who has the emergency stop button nearby. 

If you have followed the set-up correctly, Reachy is normally positioned facing the table, 20 cm backwards. So now, we turn it on and put it on its default pose (head and arms straight).

In [ ]:
reachy.turn_on()
reachy.goto_posture('default')

Now, we can define the waiting pose we want it to take before starting grasping the fruits. 

For that, we will define a function to set the joint values of every part of the robot. 

Here, we want the arms to be parallel to the ground, with the elbows back and slightly apart, grippers open. We want the head to look at its working area, pointing towards the central point 30cm in front of its end-effectors. We let the possibility to set the duration of the moves directly in the parameters of the function (by default, it's going to take 2 seconds). 

> You can set your own pose according to your environment, but be aware that there is no limitation to avoid collision between parts of the robot (right arm against left arm, arm against forearm, arm against torso, etc.). So it's your responsability to try the pose on a virtual set-up first or using the forward_kinematics() when you place the body parts where you will want them to be (the robot needs to be compliant for that : you'll have to turn it off).

For this tutorial, we will always define first the function, then execute it :

In [ ]:
import time 

def get_to_waiting_pose(reachy: ReachySDK, duration: float = 2) -> None :
    r_arm_move = reachy.r_arm.goto([30,10,-15,-115,0,0,-15], duration)
    l_arm_move = reachy.l_arm.goto([30,-10,15,-115,0,0,15], duration)

    reachy.r_arm.gripper.open()
    reachy.l_arm.gripper.open()

    waiting_position = reachy.r_arm.forward_kinematics([30,10,-15,-115,0,0,-15])[:3,3]
    head_move = reachy.head.look_at(waiting_position[0]+0.3, 0, waiting_position[2], duration, wait = True)

    print("Reachy in waiting pose")

print("Function get_to_waiting_pose defined.")

Make the robot move now : 

In [ ]:
get_to_waiting_pose(reachy, duration = 3) #each move will take 3 seconds to be done, all in the same time. 


Now we are going to move Reachy forward to the edge of the table, by controlling the mobile base. We can either :
-  **use the *goto* method**: we ask it to move 0.2m forward from its reference frame. 

Be aware that this frame is set at the start-up of the robot (so when you press the start button on the mobile base station), and not when you connect to the robot. Be careful, the reference frame doesn't update, unless you call yourself the function *reachy.mobile.reset_odometry()*. That means that if you want to move again 0.2m forward after the first move, you will have to set the command with x = 0.2 + 0.2 = 0.4m. So, to be sure the reference frame is well set, we can call the reset_odometry before sending the command. 

- **use the *translate_by* method** : we ask it to move 0.2m forward from its actual position. 

This second method seems safer in our case, so you can make the robot move now : 

In [ ]:
reachy.mobile_base.reset_odometry()
reachy.mobile_base.translate_by(x= 0.2, y=0.0)

If Reachy is not close enough to the table, you can adjust yourself by calling again the *translate_by* method, or turn it off (<code>reachy.mobile_base.turn_off()</code>) and move the robot yourself to the right place. 

Now, it's set for its grasping task. Let's see how we can deal with it !

### 4.4. Construct our program

We need to construct the program that will make Reachy sort the fruits. 

For that, we need it to:  
1.    Detect the specific objects
3.    Determine the closest object to the corresponding effector (left one for the oranges, right one for the apples)
4.    Make the corresponding arm move to the closest fruit et grasp it
5.    Make it go above the target object (the plate for the oranges, the bowl for the apples) and drop it
6.    Go back to its waiting pose. 

So, let's do that step by step !

#### 4.4.1. Detect the specific objects

We need to use the Perception module to detect the oranges, the apples, the plate and the bowl, and to extract pertinent informations.

if you have followed the set-up correctly, you should have a white plate and oranges on Reachy's left side and a white bowl and apples on its right side, like in the photos below. 

<p align="center"> 
    <img src="images/Reachy_fruit_face.jpg" alt="Image face" style="width: 20%; ; margin-right: 10px;"/>
    <img src="images/Reachy_fruits_above.jpg" alt="Image above" style="width: 20%;"/>
</p>

Check on the camera view displayer that all your objects are detected and not hidden by the arms of the robot. If so, move them a bit. Fruits need to be standing up to be well detected. 


Now, you can use the function *get_objects_infos()* of Perception, to get all the informations of the extracted objects who have a prediction score above the detection threshold you set.

In [ ]:
detection_threshold = 0.2
detected_objects = perception.get_objects_infos(detection_threshold)
print(detected_objects)

As you can see, it returns a list of unsorted dictionnaries representing each object detected among all the researched objects, with different informations, as the label, the score prediction, the bounding box, the mask... 

So we need to differenciate each object, depending on its type. We will define a function to do that. 

We give as parameters the Perception instanciation, the type of fruits to be placed to the left side, to the right side, the left target object, the right target object and the detection threshold. And it will return the list of dictionnaries of oranges, of apples, and the dictionnaries of the white plate and the white bowl. 

To avoid errors, we make sure to detect the two target objects to continue : if there are not detected, we reduce the detection threshold until a minimum threshold. 

In [ ]:
from typing import Tuple, Union, List, Dict, Optional

def get_selected_objects(
    perception: Perception, 
    obj_to_left: str, 
    obj_to_right: str, 
    left_obj_target: str, 
    right_obj_target: str, 
    detection_threshold: float
) -> Tuple[List[dict], List[dict], dict, dict]:
    
    detected_objects = perception.get_objects_infos(detection_threshold)

    # extracting the fruits to be drop to the left and to the right side
    objects_to_left = [obj for obj in detected_objects if obj['name'] == obj_to_left]
    objects_to_right = [obj for obj in detected_objects if obj['name'] == obj_to_right]

    # Reduction of the detection threshold while the containers aren't detected until a minimum threshold is reached
    min_detection_threshold = 0.1
    while True : 
        left_target = [obj for obj in detected_objects if obj['name'] == left_obj_target]
        right_target = [obj for obj in detected_objects if obj['name'] == right_obj_target]

        if left_target != [] and right_target != [] :
            break
        
        if detection_threshold > min_detection_threshold : 
            detection_threshold -= 0.01
            print(f"Targets not found, lowering threshold to {detection_threshold}")
        else : 
            print("Targets not found, new try with the minimal threshold")
        detected_objects = perception.get_objects_infos(detection_threshold)
        time.sleep(1.5)
        

    print(f'{len(detected_objects)} objects detected including {len(objects_to_left)} {obj_to_left}s, {len(objects_to_right)} {obj_to_right}s, {len(left_target)} {left_obj_target} and {len(right_target)} {right_obj_target}.')
    
    return objects_to_left, objects_to_right, left_target[0], right_target[0]

print("Function get_selected_objects defined.")

Now, you can execute this function to have the list of dictionnaries of each object type. 

In [ ]:
oranges, apples, plate, bowl = get_selected_objects(perception, obj_to_left, obj_to_right, left_obj_target, right_obj_target, detection_threshold)

#### 4.4.2. Get the closest fruit to grasp

Now we want to determine the fruit that requires the less effort from Reachy to grasp, meaning the fruit that is the closest to his corresponding effector (oranges will only be grasped by the left arm, and apples by the right arm). So we will define a function for that. 

We can first define a function to calculate distance between centroids of two objects defined by a dictionnary.


In [ ]:
import numpy.typing as npt
import numpy as np

def distance_between_objects(obj1: npt.NDArray[np.float64], obj2: npt.NDArray[np.float64]) -> float : 
    return np.linalg.norm(obj1[:3,3]-obj2[:3,3])

dist = distance_between_objects(oranges[0]['pose'], plate['pose']) #returns the distance in meters between the first orange and the plate
print(f'Distance between the first orange and the plate: {dist:.2f}m.')

Then, we need to define functions to add some conditions for the object to be graspable : 

- it has not to be already in one of the target objects  : that's the goal of the *object_in_target()* function (it uses a threshold of distance between the centroids)

- it has to be different from the last grasped object : that's the goal of the *is_a_new_object()* function, it avoids trying to grasp the same object a second time because of image capture latency and uses also a threshold of distance between centroids. 

- it has to be far enough from the robot to avoid collision with its torso : that's the goal of the *is_too_close* function.

In [ ]:
def object_in_target(obj: Dict, target: Dict, threshold: float) -> bool:
    return distance_between_objects(obj['pose'], target['pose']) < threshold


def is_a_new_object(actual_obj: Dict, former_obj: Dict, threshold: float) -> bool:
    if former_obj == []: 
        return True
    else: 
        return distance_between_objects(actual_obj['pose'], former_obj['pose']) > threshold


def is_too_close(obj: Dict, x_lim: float = 0.15, y_lim: float = 0.15) -> bool:
    if obj['pose'][0,3] < x_lim and abs(obj['pose'][1,3]) < y_lim:
        print(f"Object {obj['name']} at {obj['pose'][:3,3]} is too close to the robot")
        return True
    return False

print("Functions object_in_target, is_a_new_object and is_too_close defined.")

Now, we can define our global function, that will test for each fruit the distance with its corresponding effector and the compliance to the conditions (done by an internal function). It will return the closest object as a dictionnary and the target side as a string. It will print the closest fruit and its position in Reachy's frame. If no object respects the conditions, it will return an empty list.

In [ ]:
def get_closest_object(
    reachy: ReachySDK,  
    former_object : Dict, 
    obj_to_left: List[dict], 
    obj_to_right: List[dict], 
    left_target: Dict, 
    right_target: Dict, 
    dist_threshold : float
) -> Tuple[Optional[Dict], Optional[str]]:

    def find_closest(objects, effector_pos, side_name):
        #the closest_object is an object that is not already in one of the targets, not too close to the robot and not the same as the former_object
        nonlocal closest_object, min_dist, side
        for obj in objects:
            dist_with_effector = distance_between_objects(obj['pose'], effector_pos)
            if (dist_with_effector < min_dist and 
                not object_in_target(obj, left_target, dist_threshold) and
                not object_in_target(obj, right_target, dist_threshold) and 
                not is_too_close(obj) and 
                is_a_new_object(obj, former_object, fruit_radius)):
                
                closest_object = obj
                min_dist = dist_with_effector
                side = side_name

    closest_object, side = [], None
    min_dist = np.inf
    fruit_radius = 0.02

    #Get position of the end effectors
    position_l_effector = reachy.l_arm.forward_kinematics()
    position_r_effector = reachy.r_arm.forward_kinematics()
    
    #if the containers are detected, we look for the closest object with its relative effector
    if left_target : 
        find_closest(obj_to_left, position_l_effector, 'left')

    if right_target:
        find_closest(obj_to_right, position_r_effector, 'right')

    if closest_object:
        print(f"Closest fruit is {closest_object['name']} at the coordinates {closest_object['pose'][:3,3]}")
    else: 
        print("No fruit detected in the workspace")

    return closest_object, side

print("Function get_closest_object defined.")

Now it's your turn to determine which fruit is the closest : 

In [ ]:
former_object = []
closest_object, target_side = get_closest_object(reachy, former_object, oranges, apples, plate, bowl, 0.15)

#### 4.4.3. Grasp the fruit

Now that we know which is the closest fruit, we need to make Reachy catch it. For that, we have to follow some steps : 

First, we need to determine the grasp pose for the corresponding effector from the fruit centroid.

- For the position, we use offsets for each axis for the object to be in the middle of the gripper (the origin of the effector frame is located before the gripper)

- For the rotation :
    - we want the effector to be parallel to the ground, so the rotation in y axis is -90° (0 is with effector pointing to the ground).
    - we want the effector to orient towards the object. For that, we use the bent_position, meaning the position of the effectors when Reachy has its elbows facing torso, bent at 90°. We will use the angle between the line formed by the forearm in the bent_position and the line between the robot's elbow when facing the torso, and the object to be reached. 

    <p align="center"><img src="images/angle.jpg" alt="Image angle" style="width: 20%; ; margin-right: 10px;"/></p>

This function needs to get the dictionnary of the selected object and the target side, to return the matrix 4x4 of the grasp pose. 

In [ ]:
from reachy2_sdk.utils.utils import get_pose_matrix

def get_goal_pose(object_pose_dict: Dict, target_side: str) -> npt.NDArray[np.float64]: 
    #we define the effectors position when the robot has its forearms parallel to the ground
    bent_position = np.array([0.38622, 0.22321, -0.27036]) if target_side == 'left' else np.array([0.38622, -0.22321, -0.27036])
    
    object_pose = np.copy(object_pose_dict['pose'])
    
    # for the goal position to be in the gripper center
    object_pose [0,3] -= 0.04
    object_pose [1,3] = object_pose [1,3] + 0.01 if target_side == 'left' else object_pose [1,3] - 0.01
    object_pose [2,3] -= 0.04

    #set the orientation of the effector to reach the object
    dy = object_pose[1,3] - bent_position[1]
    dx = object_pose[0,3]

    angle_x_rad = np.arctan2(dx, dy)
    angle_x = 90 - np.degrees(angle_x_rad)

    target_pose = get_pose_matrix(object_pose[:3,3], [angle_x, -90, 0])

    print(f"goal position {target_pose[:3,3]}, goal rotation {[angle_x, -90, 0]}\n")
    
    return target_pose

print("Function get_goal_pose defined.")

We are going to try it on our closest fruit to get the grasp pose needed to catch it : 

In [ ]:
grasp_pose = get_goal_pose(closest_object, target_side)
print(f'The grasp pose is  :\n {grasp_pose}')

In [ ]:
reachy.r_arm.forward_kinematics()

But if you try to go directly from the waiting pose to the grasp pose, you'll see that sometimes the trajectory may bump into the object, and the movement may need to pass through an intermediate point to reach the fruit optimally. 

Here, we are going to add a pregrasp pose, meaning a pose that is close to the fruit, to make the next movement easier and more precise to achieve its goal. 
We set it a few centimeters before the fruit, and we define new function to calculate this pose, from the grasping pose. 

In [ ]:
from scipy.spatial.transform import Rotation as R

def get_pregrasping_pose(goal_pose: npt.NDArray[np.float64], target_side : str) -> None:
    bent_position = np.array([0.38622, 0.22321, -0.27036]) if target_side == 'left' else np.array([0.38622, -0.22321, -0.27036])
    pregrasp_pose = goal_pose.copy()

    #get the x orientation of the target pose
    rotation = R.from_matrix(goal_pose[:3,:3]).as_euler('xyz', degrees=False)[0]
    
    #pregrasp pose is 8cm before the goal pose
    pregrasp_pose[0,3] -= 0.08
    # y value is calculated from the new x value and the effector orientation needed to grasp the object
    pregrasp_pose[1,3] = pregrasp_pose[0,3] * np.tan(rotation) + bent_position[1]

    return pregrasp_pose

print("Function get_pregrasping_pose defined.")

We calculate the pregrasp pose for our closest fruit : 

In [ ]:
pregrasp_pose = get_pregrasping_pose(grasp_pose, target_side)
print(f'The pregrasp pose is  : {pregrasp_pose}')

Now, we have our grasp pose and an intermediate pose to reach the object. So we can make Reachy move !

To do that, we will convert the end effector poses into joints values, using the SDK method *inverse_kinematics* : we will get the 7 joint values of the arm needed to reach this pose. This way, we are gonna be sure that the pose is reachable by the arm, and then, we'll be able to give those commands to the robot. 

In [ ]:
#we set the controled arm to the left or right arm depending on the target side
arm = reachy.l_arm if target_side == 'left' else reachy.r_arm

joints_to_pregrasp = arm.inverse_kinematics(pregrasp_pose)
joints_to_grasp = arm.inverse_kinematics(grasp_pose)

print(f"Joints to pregrasp : {joints_to_pregrasp}\n Joints to grasp : {joints_to_grasp}")

If you have an error on this command, thats means that one of the poses is not reachable by Reachy, so you can move the object on the table and retry by starting back there : 
> Execute the cell below only if you had an error above and you moved the fruits. (*select all the cell and press Ctrl + K + U*)

In [ ]:
# oranges, apples, plate, bowl = get_selected_objects(perception, obj_to_left, obj_to_right, left_obj_target, right_obj_target, detection_threshold)
# closest_object, target_side = get_closest_object(reachy, former_object, oranges, apples, plate, bowl, 0.15)
# grasp_pose = get_goal_pose(closest_object, target_side)
# pregrasp_pose = get_pregrasping_pose(grasp_pose, target_side)

# arm = reachy.l_arm if target_side == 'left' else reachy.r_arm
# joints_to_pregrasp = arm.inverse_kinematics(pregrasp_pose)
# joints_to_grasp = arm.inverse_kinematics(grasp_pose)
# print(f"Joints to pregrasp : {joints_to_pregrasp}\n Joints to grasp : {joints_to_grasp}")

Then, we give Reachy the command, thanks to the SDK method *goto()*: 
- first, it will go the pregrasp pose
- then, to the grasp pose

Make the robot move now : 

In [ ]:
move_to_pregrasp_id = arm.goto(joints_to_pregrasp, duration = 2)
move_to_grasp_id = arm.goto(joints_to_grasp, duration = 2)

Now that Reachy got its gripper around the object, we can close it to grasp the fruit. 

In [ ]:
arm.gripper.close()

Then, we can raise the arm, using the SDK method translate_by() : 

In [ ]:
arm.translate_by(0,0,0.1, duration = 1)

Perfect, we made it ! 

But as you can see, Reachy seems a bit sad, keeping its head still. We will add a head movement for Reachy to look at the fruit it wants to catch. 
We use the method *forward_kinematics()* to get the pose of the effector in Reachy's frame, and we make the head look at its position. 

Make the robot move now : 

In [ ]:
actual_pose = arm.forward_kinematics()
reachy.head.look_at(actual_pose[0,3], actual_pose[1,3], actual_pose[2,3], 2)

That's much better, don't you think ? 

We are going to define a function for it, to execute it easily, just giving it the ReachySDK and the object we want it to look at as parameters. 

In [ ]:
def move_head(reachy: ReachySDK, target_obj: Dict) -> None:
    target_pose = target_obj['pose']
    reachy.head.look_at(target_pose[0,3], target_pose[1,3], target_pose[2,3], 2)

print("Function move_head defined.")

So now we have all the building blocks we need to construct a function to link this entire sequence together.

Some adjustements have been made for all the moves to work finely together : 
- on the grasping movement, we use a blocking parameter (*wait=True*), for the gripper to close only when the effector is at the right place
- we wait until the gripper finished to close before raising the arm
- we use a blocking parameter also on the last translation for the next moves to wait for the end of all this sequence. 

In [ ]:
def move_to_grasp(reachy: ReachySDK, obj_to_catch: Dict, target_side: str) -> None:
    print("Move sequence to grasp : started.")

    move_head(reachy, obj_to_catch)

    grasp_pose = get_goal_pose(obj_to_catch, target_side)
    pregrasp_pose = get_pregrasping_pose(grasp_pose, target_side)
    
    arm = reachy.l_arm if target_side == 'left' else reachy.r_arm
    joints_to_pregrasp = arm.inverse_kinematics(pregrasp_pose)
    joints_to_grasp = arm.inverse_kinematics(grasp_pose)

    move_to_pregrasp_id = arm.goto(joints_to_pregrasp)
    move_to_grasp_id = arm.goto(joints_to_grasp, wait=True)
        
    #we wait for the gripper to end closing before raising the arm
    arm.gripper.close()
    while arm.gripper.is_moving():
        time.sleep(0.5)
    arm.translate_by(x=0, y=0, z=0.1, duration = 1)
    time.sleep(1)

    print("Move sequence to grasp : done.")

print("Function move_to_grasp defined.")

To try it, we are gonna first drop the fruit and go back to waiting position.

Make the robot move now : 

In [ ]:
translation_move = arm.translate_by(0,0,-0.1, duration = 1)
time.sleep(1)
arm.gripper.open()
while arm.gripper.is_moving():
    time.sleep(0.1)
get_to_waiting_pose(reachy, duration = 3)

Now with only 3 functions, we are going to : 
1. do a new object detection
2. select the new closest object 
3. make Reachy do the move sequence to grasp it. 

Make the robot move now : 

In [ ]:
oranges, apples, plate, bowl = get_selected_objects(perception, obj_to_left, obj_to_right, left_obj_target, right_obj_target, detection_threshold)
closest_object, target_side = get_closest_object(reachy, former_object, oranges, apples, plate, bowl, 0.15)
move_to_grasp(reachy, closest_object, target_side)

Well done ! Now let's continue to the drop sequence. 

#### 4.4.4. Drop the fruit

We are going to build a sequence similar to the previous one to drop the grasped fruit in its target object. Hopefully, we defined a lot of functions that will be helpful in this part too !

The first step is to determine the drop pose. To do that, we are gonna use the centroid of the target object, that we determined thanks to the target_side of the closest object. 

As we want it to drop the fruit on the object, and not catch the object, we are going to virtually increase the z value of the centroid, for the effector to get on top of the object and to avoid any collision. Here, we add 15 cm. 
> Of course, it will depend on the type of object you're using (something rather flat or hollow), you can adjust the value it yourself after testing it. 

In [ ]:
# set the arm and the target object depending on the side of the target
arm = reachy.l_arm if target_side == 'left' else reachy.r_arm
target_obj = plate.copy() if target_side == 'left' else bowl.copy()

#raise virtually the target object by 15cm
target_obj['pose'][2,3] += 0.15 

print(f"{target_obj['name']} at {target_obj['pose'][:3,3]}")

We make Reachy look at its new target, with the function *move_head* that we defined ealier. 

Make it move now : 

In [ ]:
move_head(reachy, target_obj)

Then, we can use the function *get_goal_pose* we defined earlier to get the drop pose as a 4x4 matrix : 

In [ ]:
drop_pose = get_goal_pose(target_obj, target_side)
print(f'The drop pose is  : {drop_pose}')

We can get the joints value needed to reach this pose using the inverse kinematics, as we saw before. That will also check if that pose is reachable for Reachy.

In [ ]:
joints_to_drop = arm.inverse_kinematics(drop_pose)
print(f"Joints to drop pose : {joints_to_drop}")

If you have an error on the cell above, that means that your target object is unreachable for Reachy's arm. If so, you can try moving the objet and execute the cell below until you don't have any more error : 

> Execute the cell below only if you had an error on the cell above and you did move the target object. (*select all the cell and press Ctrl + K + U*)

In [ ]:
# oranges, apples, plate, bowl = get_selected_objects(perception, obj_to_left, obj_to_right, left_obj_target, right_obj_target, detection_threshold)
# target_obj = plate.copy() if target_side == 'left' else bowl.copy()
# target_obj['pose'][2,3] += 0.15 
# drop_pose = get_goal_pose(target_obj, target_side)
# joints_to_drop = arm.inverse_kinematics(drop_pose)

Now, you can send the command to the robot and make it move : 

In [ ]:
arm.goto(joints_to_drop)

The effector is now on top of the object, but maybe a bit too high : we can translate it a bit closer.
> In the same way, you can adjust the translation distance to suit your environment.

Make it move now : 

In [ ]:
arm.translate_by(0,0,-0.03, duration = 1)


Now we can set the object free, by opening the gripper !

In [ ]:
arm.gripper.open()

We raise the arm a little to avoid collisions : 

In [ ]:
arm.translate_by(x=0, y=0, z=0.05, duration = 1)

And we get back to the waiting pose :

In [ ]:
get_to_waiting_pose(reachy)

And that's it, well done !

Now we can put it all together in a function. 
Some adjustments have been made for all the moves to work finely together : 
- on the dropping movement, we use a blocking parameter (*wait=True*), for the gripper to open only when the effector is at the right place
- we wait until the gripper finished to open before raising the arm
- we use a blocking parameter also on the last translation for the next moves to wait for the end of all this sequence. 

In [ ]:
def move_to_drop(reachy: ReachySDK, target_side: str) -> None:
    print("Move sequence to drop : started.")
    
    arm = reachy.l_arm if target_side == 'left' else reachy.r_arm
    target_obj = plate.copy() if target_side == 'left' else bowl.copy()
    
    target_obj['pose'][2,3] += 0.15

    move_head(reachy, target_obj)
    drop_pose = get_goal_pose(target_obj, target_side)
    joints_to_drop = arm.inverse_kinematics(drop_pose)
    move_to_predrop_id = arm.goto(joints_to_drop)
    move_to_drop_id = arm.translate_by(0,0,-0.03, duration = 1, wait = True)

    arm.gripper.open()
    while arm.gripper.is_moving():
        time.sleep(0.1)
    arm.translate_by(0,0,0.05, duration = 1, wait = True)

    print("Move sequence to drop : done")

print("Function move_to_drop defined.")

Let's try it ! Place the fruit on the table in front of Reachy. And we'll start from the beginning.
Reachy is going to detect the objects, select the closest one, grasp it and drop it in its target object. 

Make it move now : 

In [ ]:
oranges, apples, plate, bowl = get_selected_objects(perception, obj_to_left, obj_to_right, left_obj_target, right_obj_target, detection_threshold)
closest_object, target_side = get_closest_object(reachy, former_object, oranges, apples, plate, bowl, 0.15)
move_to_grasp(reachy, closest_object, target_side)
move_to_drop(reachy, target_side)

#### 4.4.5. Get back to the waiting position

We only have to use our function : 

In [ ]:
get_to_waiting_pose(reachy, duration = 3)

### 4.5. Make it work autonomously

We have all our functions working. Now we want Reachy to do all of this sequence autonomously, and sort fruits until there are none left. 
For that, we need to construct a loop and that requires to deal first with the unreachability errors that can happen. 

#### 4.5.1. Manage reachability errors

If a fruit or a target object is out of reach for Reachy's arm, there will be an error in the code when you call the inverse_kinematics() with the pose matrix of the object. To deal with that, we are going to add a handling of the exceptions.
When an error raises, we want Reachy to notify us that he can't go further. 

For that, we are gonna add a <code>try/except</code> in the loop and define a function for Reachy to make no from head. 

In [ ]:
def make_no_from_head(reachy : ReachySDK)-> None:
    print("Making a no from the head")
    reachy.head.look_at(x=0.5, y=0.15, z=0.15, duration=1)
    reachy.head.look_at(x=0.5, y=-0.15, z=0.15, duration=1)
    reachy.head.look_at(x=0.5, y=0.15, z=0.15, duration=1)
    reachy.head.look_at(x=0.5, y=0, z=-0.25, duration=1, wait=True)

print("Function make_no_from_head defined.")

You can try it on Reachy : 

In [ ]:
make_no_from_head(reachy)

When Reachy can't grasp the fruit, it will make no from head and that's it.

But when he can't reach the target object, it will already have the object in the gripper, so it also needs to put it back on the table. So we are going to define a function to organize the sequence :
- make no from head
- lower the effector 
- open the gripper
- go back to the waiting pose. 

In [ ]:
def target_object_unreachable(reachy: ReachySDK, target_side: str) -> None:
    make_no_from_head(reachy)
    arm = reachy.l_arm if target_side == 'left' else reachy.r_arm
    arm.translate_by(0,0,-0.1, wait = True)
    arm.gripper.open()
    if arm.gripper.is_moving() : 
        time.sleep(0.1)
    get_to_waiting_pose(reachy)

print("Function target_object_unreachable defined.")

#### 4.5.2. Make the loop

So now, we can build the autonomous program !

Let's put all together the different functions we define earlier, to make it work autonomously. 

Until you press the 'interrupt' button, the program will : 
- detect the objects, with the detection threshold you set (*be careful, it waits to detect the two target objects before moving*)
- select the closest fruit if there is still fruit to grasp
- move to grasp pose, unless it's not reachable (Reachy will make no from head and go back to the loop beginning)
- move to drop pose, unless it's not reachable (it will drop back the fruit and make no from head before getting back to its waiting position)
- get back to its waiting position and do it again

If there is no fruit left, the program will wait until you interrupt it. 


**So you can try different things :**
> - give a fruit to the robot and wait for it to grasp it. 
> - put a fruit far from the robot,
> - put a fruit in a reachable place but move the target object far from the robot.

In [ ]:
closest_object = []
detection_threshold = 0.2
#definition of the target object radius, used for the threshold of object_in_target function
target_object_radius = 0.12

try:
    while True:
        former_object = closest_object
        oranges, apples, plate, bowl = get_selected_objects(perception, obj_to_left, obj_to_right, left_obj_target, right_obj_target, detection_threshold)

        #get the closest object to its corresponding end-effector 
        closest_object, target_side = get_closest_object(reachy, former_object, oranges, apples, plate, bowl, target_object_radius)
        print('closest object', closest_object['name'] if closest_object else None)
        
        if closest_object : 
            print(f"target_side : {target_side} for {closest_object['name']}")

            try : 
                move_to_grasp(reachy, closest_object, target_side)

            except ValueError as e: 
                print("Grasping poses not reachable.")
                make_no_from_head(reachy)
                time.sleep(3)
                continue


            try :
                move_to_drop(reachy, target_side)
                get_to_waiting_pose(reachy)
            
            except ValueError as e: 
                print("Dropping poses not reachable.")
                target_object_unreachable(reachy,target_side)
                time.sleep(3)
        
        else : 
            print("No new object to grasp, waiting for a new one.")
            time.sleep(3)

except KeyboardInterrupt:
    pass


Once you have finished detecting fruits, you can stop the loop by pressing the "interrupt" button and press "q" on the annotated image.

Then we are going to make Reachy go step back from the table and going back to a standard pose, to be able to switch off safely.

Make it move now : 

In [ ]:
reachy.mobile_base.translate_by(-0.2,0)
time.sleep(1)
reachy.goto_posture('default', duration = 2, wait = True)

Then we turn Reachy off smoothly and disconnect from it. 

In [ ]:
reachy.turn_off()
reachy.disconnect()
print("Reachy is disconnected.")

Well done, you did it, you have trained a Reachy greengrocer !

## 5. Final tips

Now, you've learned how to build a sequence on Reachy using the SDK Client and Pollen-Vision : 
- object detection
- synchronization between Reachy's parts
- grasping movements
- combination of perception and movement


You can now use this sequence as a starting point to create other complex behaviors on the robot. Feel free to modify the movements, the duration of the movements, the order of the movements, etc. to get more familiar with the SDK, the robot and to check whether you can make Reachy do what you want it to do ! 

Here are some general tips to keep in mind to be the best at implementing complex behaviors on Reachy:

- **Always test behavior** on a fake robot before running it on the real robot! This will help you check if the behavior is doing what you expect it to do and to avoid any potential damage to the robot.
- If you are working on the real robot, make sure that it has enough space around it to move its arms and head, and **always** have the emergency shutdown nearby. Especially, make sure that the arms will not be blocked by objects such as a table as there are no safety yet preventing the robot for colliding with its environment.
- Split the behavior you wish to develop into smaller parts and implement them one by one. Once each part is working, you can combine them to create the full sequence. Go slow and test each part before moving on to the next one. 


## 6. Skip to the next tutorial ! 

Here, we've covered just a few of the methods that can be used on Reachy. To discover more ways of controlling the robot, don't hesitate to continue following the tutorials ! 

1. Reachy's awakening (with SDK only)

2. Reachy the mime (with SDK only)

**3. Reachy the greengrocer (with SDK & Pollen-Vision)**


Keep up and you'll be soon an expert to control Reachy ! 